# Downloading segmentations from <i>FIIND</i> on BrainBox

This script demonstrates how to obtain programmatically all segmentations available for FIIND MRI data in BrainBox.

## 1. The project on BrainBox

The project URL is https://brainbox.pasteur.fr/project/FIIND. The project contains 28 MRIs, 4 for each time point including P0 (day of birth), P2, P4, P8, P16, P32 and adult. These MRIs have a brain mask in the annotation layer `Atlas`. Finally, there are 2 text annotations: `MRI Quality` and `Comments`.

## 2. Imports

In [1]:
from __future__ import unicode_literals

# if necessary, install tqdm
#!pip install tqdm 

import requests as urlreq
import re
import os
from tqdm import tqdm

## 3. Configuration

In [2]:
# Project name
project = 'FIIND'

# Names of the segmentations to download
masknames = ['Atlas']

# Local directory for saving the segmentations
RAWSEG_DIR = './data-fiind-brainbox/'

## 4. Get information

In [3]:
# Create local download directory if it doesn't exist
if not os.path.exists(RAWSEG_DIR):
    os.makedirs(RAWSEG_DIR)

# Project URL
prj_url = "https://brainbox.pasteur.fr/project/json/" + project + "/"

# Base MRI URL
mri_url = "https://brainbox.pasteur.fr/mri/json?url="

# Download project description
res = urlreq.get( prj_url )
prj = res.json()

# Get number of files
nfiles = len( prj['files']['list'] )
print("Segmentations for %s files will be downloaded"%(nfiles))

Segmentations for 28 files will be downloaded


## 5. Download segmentations

In [4]:
# Iterate over all project files
noresult = True
for isub in tqdm(range( 0, nfiles )):
    
    # Download MRI description
    brainbox_url = mri_url + prj['files']['list'][isub]
    res = urlreq.get( brainbox_url )
    mri = res.json()
    subID = mri['name']

    # Search for the mask names atlases within the annotations
    natlases = len(mri['mri']['atlas'])
    base = "https://brainbox.pasteur.fr"+mri['url']
    
    # Iterate over all atlases
    for i in range( 0, natlases ):
        atlas_i = mri['mri']['atlas'][i]
        
        # Iterate over all requested annotation layers
        for j in masknames:
            if (
                'name' in atlas_i and
                atlas_i['name'] == j and
                atlas_i['project'] == project
            ):
                filename = atlas_i['filename']
                
                # Download the atlas
                atlas_url = base + filename
                res = urlreq.get( atlas_url )
                if res.status_code != 200:
                    # The atlas is not found
                    #print("Atlas file '{}' not found.".format(atlas_url))
                    continue
                    
                # Create a BIDS conforming filename
                myname = "sub-" + subID + "_mask-" + j + ".nii.gz"

                # Create a BIDS conforming subject directory
                dst = '{dir}/{sub}'.format( dir=RAWSEG_DIR, sub=subID)
                if not os.path.exists( dst ):
                    os.makedirs( dst )

                local_file = open( dst + "/" + myname, "wb" )
                local_file.write( res.content )
                local_file.close()
                noresult = False
                
if noresult:
    print("No atlas contained the requested masks")

100%|██████████| 28/28 [00:07<00:00,  3.62it/s]
